[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/4b_82.ipynb)

In [46]:
# https://keras.io/
!pip install -q keras
import keras

In [47]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [48]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [49]:
# Hyperparameters
batch_size = 35
num_classes = 10
epochs = 50
#l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [50]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32
32


In [51]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [52]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [53]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [54]:
num_filter = 16
dropout_rate = 0.2
l = 16
input = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)


3


In [55]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 32, 32, 16)   432         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_125 (BatchN (None, 32, 32, 16)   64          conv2d_125[0][0]                 
__________________________________________________________________________________________________
activation_125 (Activation)     (None, 32, 32, 16)   0           batch_normalization_125[0][0]    
__________________________________________________________________________________________________
conv2d_126

__________________________________________________________________________________________________
dropout_174 (Dropout)           (None, 4, 4, 8)      0           conv2d_178[0][0]                 
__________________________________________________________________________________________________
concatenate_162 (Concatenate)   (None, 4, 4, 24)     0           concatenate_161[0][0]            
                                                                 dropout_174[0][0]                
__________________________________________________________________________________________________
batch_normalization_178 (BatchN (None, 4, 4, 24)     96          concatenate_162[0][0]            
__________________________________________________________________________________________________
activation_178 (Activation)     (None, 4, 4, 24)     0           batch_normalization_178[0][0]    
__________________________________________________________________________________________________
conv2d_179

In [56]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [57]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
23905/50000 [=============>................] - ETA: 3:43 - loss: 1.7430 - acc: 0.3467

50000/50000 [==============================] - 432s 9ms/step - loss: 1.5645 - acc: 0.4225 - val_loss: 1.7584 - val_acc: 0.4639
Epoch 2/50
 5705/50000 [==>...........................] - ETA: 5:43 - loss: 1.2637 - acc: 0.5513

49980/50000 [============================>.] - ETA: 0s - loss: 1.1551 - acc: 0.5854

50000/50000 [==============================] - 411s 8ms/step - loss: 1.1552 - acc: 0.5853 - val_loss: 2.1401 - val_acc: 0.4419
Epoch 3/50
11340/50000 [=====>........................] - ETA: 5:00 - loss: 1.0273 - acc: 0.6300

50000/50000 [==============================] - 411s 8ms/step - loss: 0.9953 - acc: 0.6439 - val_loss: 1.1305 - val_acc: 0.6238
Epoch 4/50
  945/50000 [..............................] - ETA: 6:22 - loss: 0.9690 - acc: 0.6614

...

49980/50000 [============================>.] - ETA: 0s - loss: 0.2920 - acc: 0.8963

50000/50000 [==============================] - 413s 8ms/step - loss: 0.2922 - acc: 0.8962 - val_loss: 0.5684 - val_acc: 0.8375
Epoch 49/50
11305/50000 [=====>........................] - ETA: 5:02 - loss: 0.2868 - acc: 0.9005

50000/50000 [==============================] - 413s 8ms/step - loss: 0.2873 - acc: 0.8985 - val_loss: 0.6170 - val_acc: 0.8371
Epoch 50/50
  910/50000 [..............................] - ETA: 6:24 - loss: 0.2818 - acc: 0.8956

49980/50000 [============================>.] - ETA: 0s - loss: 0.2888 - acc: 0.8964

50000/50000 [==============================] - 414s 8ms/step - loss: 0.2889 - acc: 0.8964 - val_loss: 0.6692 - val_acc: 0.8222


In [58]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 24s 2ms/step
Test loss: 0.6692031421899796
Test accuracy: 0.8222


In [59]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [60]:
#from google.colab import files

#files.download('DNST_model.h5')